<a href="https://colab.research.google.com/github/amygeorge/flask_app/blob/master/Assignment_5_Advanced_Text_Classification_(CPSC_436N).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# CPSC 436N - Assignment 5
# Bebo Elhosary 20375168
# Amy George 18392134
---

# Assignment 5: Advanced Text Classification - Transformers and Syntactic Features (CPSC 436N)

This assigment can be seen as a continuation of assigment 3. You will still work on the task of text classification by while in assignment 3 you studied 
 a Bag Of Word (BOW), and a word embedding-based classifiers, in this assigment you will develp a text classifier based on a  pretrained language model (PrLM) and one based on syntactic (POS and dependency) features.

You will test these implemented classifiers on subsets of the 20newsgroup corpus, and analyze the errors and successes of one model compared to the other.


## 1. Install Spacy and language models will be used in later steps:

#### Install Spacy v3.0: 


In [ ]:
import sys

!{sys.executable} -m pip install spacy==3.0

     |████████████████████████████████| 12.7 MB 5.8 MB/s 
     |████████████████████████████████| 9.1 MB 17.1 MB/s 
     |████████████████████████████████| 653 kB 7.7 MB/s 
     |████████████████████████████████| 451 kB 36.8 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


#### Download and install two trained English pipelines ([en_core_web_lg](https://spacy.io/models/en) and [en_core_web_trf](https://spacy.io/models/en)) provided by Spacy: 

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 778.8 MB 20 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 459.7 MB 17 kB/s 
     |████████████████████████████████| 42 kB 837 kB/s 
     |████████████████████████████████| 1.1 MB 9.6 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 40 kB 5.1 MB/s 
     |████████████████████████████████| 2.6 MB 37.3 MB/s 
     |████████████████████████████████| 3.3 MB 28.2 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [12]:
#import libs needed for utilizing GPU.
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu
set_gpu_allocator("pytorch")

require_gpu(0)

import torch
#import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
 device = torch.device("cuda") 
 print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
 print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load dataset (20newsgroups) (same as assignemt 3)

For this assignment, we train and test classification models on the 20newsgroups corpus, which can be easily fetched by sklearn. This dataset comprises around 18000 newsgroups posts on 20 topics and has been splited into two subsets by sklearn for model training and testing. 

To ensure this assignment is manageable and won't take too long for training and inference, we will use a subset of 20newsgroups only covering samples belonging to either one of the two classes (like '__talk.politics.misc__', '__talk.religion.misc__' used below). With this setting, we simplify the multiclass classification to binary classification. 

If you need a refresher from assignment3, please refer the following two links for more details about 20newsgroups corpus and how to load and process it with sklearn:

* http://qwone.com/~jason/20Newsgroups/
* https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [13]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', categories=['talk.politics.misc','talk.religion.misc'])
newsgroups_test = fetch_20newsgroups(subset='test', categories=['talk.politics.misc','talk.religion.misc'])

## 3. Tokenizer is created with Spacy as in assignment-3:

Tokenization is a critical preprocessing step when we work with text data. What [tokenization](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/) does is separating input text into tokens, which can be either words, characters, or subwords, and further processing these tokens to reduce the noise caused by informal expressions and typos contained in text.

There are different ways to customize your tokenization strategy. Some typical steps include __lowercasing__, __stop words and punctuations removing__, __lemmatization__ and __filtering tokens with part-of-speech tagging__.

In the code cell below, the tokenization function with Spacy (named as __spacy_tokenizer__) covers:

* Lowercasing

* Removing stop words

* Removing punctuations

* Lemmatization


In [14]:
#import libs needed for tokenization.
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

nlp = spacy.load('en_core_web_lg')  # Load trained English pipeline "en_core_web_lg".
punctuations = string.punctuation  # Get the list of punctuation.
stop_words = spacy.lang.en.stop_words.STOP_WORDS  # Get the list of stop words identified in the loaded language model.

# Creating your own tokenizer function with functions built in Spacy.
def spacy_tokenizer(doc):

    tokens = nlp(doc)  # Splits the doc into tokens and applies the loaded pipeline 
    #(create all linguistic annotations for the doc, including POS etc).

    # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() for word in tokens ]

    # Removing stop words
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuations ]
    
    return tokens  # return preprocessed list of tokens.

## 4. Build the pipeline for the classification model with pretrained language model

Different from the word embedding-based classifier we implemented in Assignment3, here we replace the document vectorizer with the pretrained neural language model "en_core_web_trf" (RoBERTa) provided by Spacy. You can learn more about this language model from [RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/pdf/1907.11692.pdf).

While in Assignment3 we used the SpacyWordEmb_CPNT in the pipeline, now we need to  implement the class of document vectorization component with pretrained language model (named as __SpacyLangModel_CMPT__) and place it into the pipeline.

**Q1:** Please where requested add comments to the transform function in SpacyLangModel_CMPT in the code cell below. The comments should explain the best you can what the code is doing and why.

You may find the link below is useful when understanding how to use pretrained language models in Spacy:

https://spacy.io/usage/embeddings-transformers


In [15]:
#Now let's design the pipeline for the text classification model with sklearn!
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# import libs and classes needed for component construction.
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import cupy as cp

# define the class of the preprocessing component (same as in assignment 3)
class Preprocessing_CPNT(BaseEstimator, TransformerMixin):
    def transform(self, X, **transform_params): # the function actually performs the preprocessing, X contains all samples in the input corpus.
            return [spacy_tokenizer(text) for text in X] # apply the spacy tokenizer we implemented above to preprocess all input data.

    # in this case, we don't need to calculate values for later scaling, so fit() is doing nothing but just return self.
    def fit(self, X, y=None, **fit_params):
        return self

# set classifier to Logistic Regression        
classifier = LogisticRegression(max_iter = 1000)

#import libs needed for utilizing GPU.
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu

# define the class of the document vectorization component with pretrained language model
class SpacyLangModel_CMPT(BaseEstimator, TransformerMixin):
    def __init__(self, nlp): # define the language model and dimension of embeddings we want to use.
        self.nlp = nlp
        self.dim = 768
        
    def transform(self, X): # the function actually vectorizes samples with pretrained neural language model, X contains all samples in the input corpus.
        X_str = []
        for text in X:
            X_str.append(' '.join(text)) #ADD COMMENT HERE: takes an array of tokens, and converts them to a space seperated sentence of tokens
        return [cp.asnumpy(self.nlp(text)._.trf_data.tensors[0][0][0]) for text in X_str] #ADD COMMENT HERE: represents every sentence as a
                # document's vector or as a vector of BPE token-pieces
      
    
    # in this case, fit() is doing nothing.
    def fit(self, X, y=None):
        return self

nlp = spacy.load("en_core_web_trf")  # Load the pretrained english pipeline (language model) "en_core_web_trf"

# Create pipeline for the classfier with pretrained language model
pipe = Pipeline([("preprocessing", Preprocessing_CPNT()),
                ('encoding', SpacyLangModel_CMPT(nlp)),
                ("classifier", classifier)])

Now let's train this classification model based on pretrained language model.

Please read:

https://spacy.io/usage/embeddings-transformers

for how to use GPU for model training and inference.

In [16]:


# # Use GPU if available
# if prefer_gpu():
#   set_gpu_allocator("pytorch")
#   require_gpu(0)
  
# else:
#   print("GPU unavailable, we do not recommend running this on a CPU.")


# Model Training (This may take > 5 min depending on the GPU)
pipe.fit(newsgroups_train.data, newsgroups_train.target)

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors


Pipeline(steps=[('preprocessing', Preprocessing_CPNT()),
                ('encoding',
                 SpacyLangModel_CMPT(nlp=<spacy.lang.en.English object at 0x7f68ca572b90>)),
                ('classifier', LogisticRegression(max_iter=1000))])

And do prediction on the testing set and evaluate the performance of this classifier.

In [17]:
from sklearn import metrics
# Predicting with a test dataset
predicted_trf = pipe.predict(newsgroups_test.data)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(newsgroups_test.target, predicted_trf))

Logistic Regression Accuracy: 0.8128342245989305


## 5. Build the pipeline for the syntax/semantic-based classifier

Here we explore how to vectorize documents with two syntatic features: part-of-speech tags and syntactic dependencies. 

**Q2:** Why do we create the pipeline with only document vectorization and classifier component here? In other words why don't we want the spacy tokenizer implemented above to be run before we do POS tagging and dependency parsing?

**Answer: We don’t run the tokenizer because we want to keep the grammar and structure of the text. Otherwise, important information for POS tagging and dependency parsing will be lost.** 

Now you need to implement the syntax-based document vectorization component (named as __Syntax_CMPT__) to extract pos and dependency features and use them to represent the document. 

**Q3:** Please where requested add comments to the transform function in __Syntax_CMPT__ in the code cell below. The comments should explain the best you can what the code is doing and why.


In [18]:
class Syntax_CMPT(BaseEstimator, TransformerMixin):
    def __init__(self, nlp): # define the language pipeline we want to use and vectorizers for POS and dependency respectively.
        self.nlp = nlp
        self.pos_vectorizer = CountVectorizer()
        self.dep_vectorizer = CountVectorizer()
        
    def transform(self, X): # transform the testing data into pos+dep vectors with the pos and dependency vectorizer learned 
    #from training data.
        pos_doc_list = []; dep_doc_list = [];
        for text in X:
            doc = self.nlp(text) # apply the loaded language pipeline to testing samples.
            doc_pos = []; doc_dep = [];
            for sent in doc.sents: #ADD COMMENT HERE: go through all the sentences in the document
              doc_pos = doc_pos+[str(token.tag_) for token in self.nlp(str(sent))] #ADD COMMENT HERE: go through all the tokens in the sentence
              # and append the token's tag (pos) to current doc's list of pos
              doc_dep = doc_dep+[str(token.dep_) for token in self.nlp(str(sent))] #ADD COMMENT HERE go through all the tokens in the sentence
              # and append the token's dependency raltion tag to current doc's list of dep tags
            pos_doc_list.append(' '.join(doc_pos)) # turn pos tag list into a string.
            dep_doc_list.append(' '.join(doc_dep)) # turn dependecy relation list into a string.

        doc_num = len(pos_doc_list)  

        X_pos = self.pos_vectorizer.transform(pos_doc_list).toarray() # transform samples into pos vectors.
        X_dep = self.dep_vectorizer.transform(dep_doc_list).toarray() # transform samples into dep vectors.

        return np.array([list(X_pos[i]) + list(X_dep[i]) for i in range(doc_num)]) # concatenate pos and dependency vectors.
    

    def fit(self, X, y=None): # different from classifiers above, this time we need to first use fit() to learn values for pos and dependency 
    #vectorizer from the training data. 
        pos_doc_list = []; dep_doc_list = [];
        for text in X:
            doc = self.nlp(text) # apply the loaded language pipeline to training samples.
            doc_pos = []; doc_dep = [];
            for sent in doc.sents: 
              doc_pos = doc_pos+[str(token.tag_) for token in self.nlp(str(sent))]
              doc_dep = doc_dep+[str(token.dep_) for token in self.nlp(str(sent))] 
            pos_doc_list.append(' '.join(doc_pos)) # turn pos tag list into a string.
            dep_doc_list.append(' '.join(doc_dep)) # turn dependecy relation list into a string.

        doc_num = len(pos_doc_list)  

        X_pos = self.pos_vectorizer.fit_transform(pos_doc_list).toarray() # fit the pos vectorizer on training data.
        X_dep = self.dep_vectorizer.fit_transform(dep_doc_list).toarray() # fit the dependency vectorizer on training data.
        
        return self

nlp = spacy.load("en_core_web_lg")  # Load the english pipeline "en_core_web_lg"

# Create pipeline for the syntax-based classifier
pipe = Pipeline([('encoding', Syntax_CMPT(nlp)),
                ("classifier", classifier)])

Now let's train this classification model based on Part-Of-Speech and dependency features.

In [19]:
# Model Training (This may take > 50 min depending on the GPU)
pipe.fit(newsgroups_train.data, newsgroups_train.target)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('encoding',
                 Syntax_CMPT(nlp=<spacy.lang.en.English object at 0x7f68c84e0850>)),
                ('classifier', LogisticRegression(max_iter=1000))])

And do prediction on the testing set and evaluate the performance of this classifier.

In [20]:
from sklearn import metrics

# Predicting with a test dataset (This may take > 15 min depending on the GPU)
predicted_syntax = pipe.predict(newsgroups_test.data)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(newsgroups_test.target, predicted_syntax))

Logistic Regression Accuracy: 0.696969696969697


## 6. Evaluate classifiers with more sophisticated metrics

In the steps above, we have evaluated the performances of the two classifiers only with the accuracy score.

**Q4**: Now you will have to edit the code cells for model evaluation above to add two more evaluation metrics: **Confusion matrix** and **F1 Score** (see textbook section 4.7)

Please report and discuss any  insights and observations from your analysis of the results you obtained based on these metrics.

In [21]:

from sklearn.model_selection import cross_val_score, cross_validate 
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_precision_recall_curve, precision_score, recall_score, f1_score, classification_report
# Accuracy score
print("Logistic Regression Accuracy:",metrics.accuracy_score(newsgroups_test.target, predicted_syntax))

# Precision = TP/(TP + FP)
print("Logistic Regression Precision:",metrics.precision_score(newsgroups_test.target, predicted_syntax))

# Recall = TP/(TP + FN)
print("Logistic Regression Recall:",metrics.recall_score(newsgroups_test.target, predicted_syntax))


# F1 = (2*precision*recall)/(precision+recall)
print("Logistic Regression F1:",metrics.f1_score(newsgroups_test.target, predicted_syntax))

# confusion matrix on the data
print("Logistic Regression CM:",metrics.confusion_matrix(newsgroups_test.target, predicted_syntax))

Logistic Regression Accuracy: 0.696969696969697
Logistic Regression Precision: 0.6832579185520362
Logistic Regression Recall: 0.601593625498008
Logistic Regression F1: 0.6398305084745762
Logistic Regression CM: [[240  70]
 [100 151]]
